In [2]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler

class CancerClassifier:
    def __init__(self):
        self.breast_labels = None
        self.breast_train = None
        self.breast_validate = None
        
    def read_data(self):
        # Wczytanie danych (przykładowa implementacja)
        self.breast_labels = pd.read_csv("dataset/breast-cancer.labels", header=None)[0].tolist()
        self.breast_train = pd.read_csv("dataset/breast-cancer-train.dat", header=None)
        self.breast_validate = pd.read_csv("dataset/breast-cancer-validate.dat", header=None)
        self.breast_train.columns = self.breast_labels
        self.breast_validate.columns = self.breast_labels
    
    def _prepare_data(self, df, scaler=None):
        """Przygotowanie danych: usunięcie ID, mapowanie targetu, normalizacja, dodanie biasu"""
        # Kopia danych
        data = df.copy()
        
        # Usunięcie kolumny z ID pacjenta
        if 'patient ID' in data.columns:
            data = data.drop(columns=['patient ID'])
        
        # Mapowanie wartości docelowej
        data['Malignant/Benign'] = data['Malignant/Benign'].map({'M': 1, 'B': 0})
        
        # Separacja cech i etykiet
        y = data['Malignant/Benign'].values
        X = data.drop(columns=['Malignant/Benign']).values
        
        # Normalizacja
        if scaler is None:
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
        else:
            X_scaled = scaler.transform(X)
        
        # Dodanie kolumny biasu (x0 = 1)
        X_final = np.c_[np.ones(X_scaled.shape[0]), X_scaled]
        
        return X_final, y, scaler
    
    def _compute_learning_rate(self, X):
        """Obliczenie stałej uczącej na podstawie wartości własnych"""
        A = X.T @ X
        eigenvalues = np.linalg.eigvals(A)
        lambda_max = np.max(eigenvalues)
        return 1.0 / lambda_max
    
    def gradient_descent(self, X, y, alpha, max_iters=10000, tol=1e-6):
        """Implementacja spadku gradientowego"""
        n_samples, n_features = X.shape
        beta = np.zeros(n_features)  # Inicjalizacja wag
        cost_history = []
        start_time = time.time()
        
        for i in range(max_iters):
            # Obliczenie predykcji i błędu
            y_pred = X @ beta
            error = y_pred - y
            
            # Obliczenie kosztu (MSE)
            cost = np.sum(error**2) / (2 * n_samples)
            cost_history.append(cost)
            
            # Obliczenie gradientu
            gradient = (X.T @ error) / n_samples
            
            # Aktualizacja wag
            beta_new = beta - alpha * gradient
            
            # Sprawdzenie warunku stopu
            if np.linalg.norm(beta_new - beta) < tol:
                beta = beta_new
                break
                
            beta = beta_new
            
            # Warunek stopu na podstawie zmiany kosztu
            if i > 0 and abs(cost_history[-1] - cost_history[-2]) < tol:
                break
        
        time_elapsed = time.time() - start_time
        return beta, cost_history, i+1, time_elapsed
    
    def least_squares(self, X, y):
        """Rozwiązanie metodą najmniejszych kwadratów"""
        start_time = time.time()
        beta = np.linalg.pinv(X) @ y  # Użycie pseudoodwrotności
        time_elapsed = time.time() - start_time
        return beta, time_elapsed
    
    def predict(self, X, beta, threshold=0.5):
        """Predykcja etykiet na podstawie wag"""
        y_continuous = X @ beta
        y_pred = (y_continuous >= threshold).astype(int)
        return y_pred
    
    def accuracy(self, y_true, y_pred):
        """Obliczenie dokładności"""
        return np.mean(y_true == y_pred)
    
    def run(self):
        # Wczytanie danych
        self.read_data()
        
        # Przygotowanie danych treningowych
        X_train, y_train, scaler = self._prepare_data(self.breast_train)
        
        # Przygotowanie danych walidacyjnych
        X_val, y_val, _ = self._prepare_data(self.breast_validate, scaler)
        
        # Obliczenie stałej uczącej dla GD
        alpha = self._compute_learning_rate(X_train)
        print(f"Obliczona stała ucząca: {alpha:.6f}")
        
        # Trenowanie Gradient Descent
        beta_gd, cost_history, iters, time_gd = self.gradient_descent(
            X_train, y_train, alpha
        )
        
        # Trenowanie Least Squares
        beta_ls, time_ls = self.least_squares(X_train, y_train)
        
        # Predykcje i ocena GD
        y_pred_gd = self.predict(X_val, beta_gd)
        acc_gd = self.accuracy(y_val, y_pred_gd)
        
        # Predykcje i ocena LS
        y_pred_ls = self.predict(X_val, beta_ls)
        acc_ls = self.accuracy(y_val, y_pred_ls)
        
        # Wyniki
        print("\n" + "="*50)
        print("Wyniki porównawcze metod")
        print("="*50)
        print(f"{'Metoda':<25} | {'Dokładność':<12} | {'Czas [s]':<10} | Iteracje")
        print("-"*50)
        print(f"{'Gradient Descent':<25} | {acc_gd:.6f}    | {time_gd:.6f}    | {iters}")
        print(f"{'Least Squares':<25} | {acc_ls:.6f}    | {time_ls:.6f}    | -")
        print("="*50)
        
        # Analiza złożoności
        n, p = X_train.shape
        print("\nTeoretyczna złożoność obliczeniowa:")
        print("Gradient Descent: O(k * n * p)")
        print("Least Squares: O(n * p² + p³)")
        
        return {
            'gd': {'beta': beta_gd, 'accuracy': acc_gd, 'time': time_gd, 'iterations': iters},
            'ls': {'beta': beta_ls, 'accuracy': acc_ls, 'time': time_ls}
        }


# Uruchomienie klasyfikatora
if __name__ == "__main__":
    classifier = CancerClassifier()
    results = classifier.run()

Obliczona stała ucząca: 0.000256

Wyniki porównawcze metod
Metoda                    | Dokładność   | Czas [s]   | Iteracje
--------------------------------------------------
Gradient Descent          | 0.976923    | 3.464080    | 8527
Least Squares             | 0.973077    | 0.002167    | -

Teoretyczna złożoność obliczeniowa:
Gradient Descent: O(k * n * p)
Least Squares: O(n * p² + p³)
